In [0]:
from pyspark.sql import functions as F, types as T

rows_customers = [
    (1, "Asha", "IN", True),
    (2, "Bob", "US", False),
    (3, "Chen", "CN", True),
    (4, "Diana", "US", None),
    (None, "Ghost", "UK", False), # NULL key to demo join behavior
]

rows_orders = [
    (101, 1, 120.0, "IN"),
    (102, 1, 80.0, "IN"),
    (103, 2, 50.0, "US"),
    (104, 5, 300.0, "DE"), # no matching customer_id
    (105, 3, 200.0, "CN"),
    (106, None, 15.0, "UK"), # NULL key wont match
    (107, 3, 40.0, "CN"),
    (108, 2, 75.0, "US"),
]

schema_customers = T.StructType([
    T.StructField("customer_id", T.IntegerType(), True),
    T.StructField("name", T.StringType(), True),
    T.StructField("country", T.StringType(), True),
    T.StructField("vip", T.BooleanType(), True),
])

schema_orders = T.StructType([
    T.StructField("order_id", T.IntegerType(), True),
    T.StructField("customer_id", T.IntegerType(), True),
    T.StructField("amount", T.DoubleType(), True),
    T.StructField("country", T.StringType(), True), #same column name to show collisions
])

df_customers = spark.createDataFrame(rows_customers, schema_customers)
df_orders = spark.createDataFrame(rows_orders, schema_orders)

display(df_customers)
display(df_orders)


In [0]:
df_inner = df_orders.join(df_customers, on="customer_id", how="right")
display(df_inner)

In [0]:
o = df_orders.alias("o")
o.show()

In [0]:
o, c = df_orders.alias("o"), df_customers.alias("c")

df_inner = o.join(c, on="customer_id", how="inner")
display(df_inner)

In [0]:
df_inner_clean = (
    o.join(c, on="customer_id", how="inner")
    .select("order_id", "customer_id", "amount", 
            F.col("o.country").alias("ship_country"), 
            "name", 
            F.col("o.country").alias("cust_country"), 
            "vip")
)

display(df_inner_clean)

In [0]:
display(o.join(c, on="customer_id", how="full"))

In [0]:
display(o.join(c, on="customer_id", how="left_semi")) # orders with a known customer

In [0]:
display(o.join(c, on="customer_id", how="left_anti")) # orphan orders (no matching customer)

In [0]:
df_multi = o.join(c, on=["customer_id", "country"], how="inner")
display(df_multi)